For 2 languages Hindi and English

In [ ]:
import numpy as np
import nltk
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import pickle
nltk.download('stopwords')

In [7]:
file = open("Hindi_Stopwords.txt", encoding = "UTF-8")
stopWords = file.read().split("\n")

In [8]:
paras_e = open('Geeta_English.txt', encoding="UTF8").read().split("\n")
paras_h = open('gita_Hindi.txt', encoding="UTF8").read().split("\n")


Stemming and tokensing Hindi words

In [11]:
from indicnlp.tokenize import indic_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [20]:
suffixes = {
    1: ["ो", "े", "ू", "ु", "ी", "ि", "ा"],
    2: ["कर", "ाओ", "िए", "ाई", "ाए", "ने", "नी", "ना", "ते", "ीं", "ती", "ता", "ाँ", "ां", "ों", "ें"],
    3: ["ाकर", "ाइए", "ाईं", "ाया", "ेगी", "ेगा", "ोगी", "ोगे", "ाने", "ाना", "ाते", "ाती", "ाता", "तीं", "ाओं", "ाएं", "ुओं", "ुएं", "ुआं"],
    4: ["ाएगी", "ाएगा", "ाओगी", "ाओगे", "एंगी", "ेंगी", "एंगे", "ेंगे", "ूंगी", "ूंगा", "ातीं", "नाओं", "नाएं", "ताओं", "ताएं", "ियाँ", "ियों", "ियां"],
    5: ["ाएंगी", "ाएंगे", "ाऊंगी", "ाऊंगा", "ाइयाँ", "ाइयों", "ाइयां"],
}

def stemming_hindi(text):
    for L in 5, 4, 3, 2, 1:
        if len(text) > L + 1:
            for suf in suffixes[L]:
                if text.endswith(suf):
                    return text[:-L]
    return text

In [21]:
def preprocess_text(string, stopWords):
    punctuations=["।",";",",",":","!",'"',"?",":-","-","{","(","}",")","_","०","S","―","=","[","]","......",":-",".","॥",'”',"|","“","'"]
    string = "".join([w if w not in punctuations else " " for w in string])  #To remove punctuations
    tokens = string.split()
    tokens = [stemming_hindi(word) for word in tokens if word not in stopWords]
    tokens = [w for w in tokens if w not in stopWords]
    return tokens

In [30]:
pos_lis = {}
for idx, para in enumerate(paras_h):
    words = preprocess_text(para, stopWords)
    
    for word in words:
        if word in pos_lis.keys():
            if idx in pos_lis[word].keys():
                pos_lis[word][idx] += 1
            else:
                pos_lis[word][idx] = 1
        else:
            temp = {idx : 1}
            pos_lis[word] = temp.copy()

In [31]:
with open('data.pkl', 'wb') as file:
    pickle.dump(pos_lis, file, protocol=pickle.HIGHEST_PROTOCOL)

Same way implementing for BM25 model

In [37]:
def BM25(query, pos_lisl, l = 5, b = 0.75, k = 2):
    q_tokens = preprocess_text(query,stopWords)
    lengths = {}
    N = len(paras_e)
    avg_len = 0
    for idx, para in enumerate(paras_e):
        lengths[idx] = len(para)             #cal no of words of each file
        avg_len += lengths[idx]
    avg_len /= N
    #Calculate idf of each token
    idf = {}
    for word in np.unique(q_tokens):
        if word in pos_lisl.keys():
            df = len(pos_lisl[word].keys())
        else:
            df = 0
        idf[word] = np.log((N - df + 0.5) / (df + 0.5))
    score = {}
    for idx, para in enumerate(paras_e):
        s = 0
        for word in np.unique(q_tokens):
            tf = 0
            if word in pos_lisl.keys() and idx in pos_lisl[word].keys():
                tf = pos_lisl[word][idx]
            s += idf[word] * (tf * (k + 1)) / (k*(1 - b + b*lengths[idx]/avg_len) + tf)
        score[idx] = s
    return sorted(score, key = score.get, reverse=True)[:l]

Searching and Evaluating 

In [38]:
from googletrans import Translator

def get_translation(data, dest):
    translator = Translator()
    text = translator.translate(data, dest).text
    return text

In [45]:
slokas = open("Shlokas.txt",encoding="UTF-8").read().split("\n")

In [46]:
def search(query, pos_lis, lang, l = 5):
    query = get_translation(query, "hi")
    tokens = preprocess_text(query, stopWords)
    
    query = " ".join(tokens)

    indxs = BM25(query, pos_lis, l)
    print(indxs)
    if lang == 'e':
        return [f'{slokas[idx]}{paras_e[idx]}' for idx in indxs], indxs
    else:
        return [f'{slokas[idx]}{paras_h[idx]}' for idx in indxs], indxs

In [49]:
query ="That gives itself to follow shows of sense"
search(query,pos_lis,"h")

[143, 167, 193, 100, 112]


(['अर्जुन उवाच ।हे शत्रुओं का नाश करने वाले, जो पूजा के पात्र हैं, मैं अपने बाणों से युद्ध करूंगा।',
  'सेनयोरुभयोर्मध्ये विषीदन्तमिदं वचः ॥ जो तुम्हारे लिए शोक नहीं करते उनके लिए तुम शोक नहीं करते और ज्ञान की बातें कहते हो',
  'न जायते म्रियते वा कदाचिन्वह जो सदा के लिए वेदों को नष्ट कर देता है, यह अजन्मा और अविनाशी है।',
  'यद्यप्येते न पश्यन्ति लोभोपहतचेतसः ।परिवार को नष्ट करना पाप है और मित्र को धोखा देना पाप है',
  'ये दोष कुल का नाश करने वाले और जातिगत भ्रम पैदा करने वाले होते हैं।'],
 [143, 167, 193, 100, 112])